# RA Scrape Cleaning Notebook

This notebook is structured to clean the data collected from ra_scraper.py

In [1]:
import pandas as pd
from tqdm import tqdm
import re
import os

### Loading the data

The data was collected by a scraper and saved in spreadsheets named after the club. The goal here is to concat all of these .xlsx files into a Pandas DataFrame, clean DJ names, and create a new dataset that is a count of each DJ's appearance at a fest/venue/party.

In [2]:
folder = '/home/xristos/projects/ra_calendar_scraper/data'
df_list = []
fest_list = []
festivals = ['time warp','dekmantel','junction 2','glitch festival','awakenings festival',
             'sonus festival','kappa futurfestival','love family park','dgtl']
#loops through the folder and creates a dataframe from each spreadsheet
for file in os.listdir(folder):
    if file.split('.xlsx')[0].strip() in festivals:
        df = pd.read_excel(folder+f"/{file}")
        fest_list.append(df.drop_duplicates())
    else:
        df = pd.read_excel(folder+f"/{file}")
        df_list.append(df.drop_duplicates())

## Creating a list of the club and fest names
These lists are going to be used as a reference when filtering the data

In [3]:
#the file names were named after the search term used to get the data
club_names = os.listdir(folder)
club_names = [x.split('.xlsx')[0].strip() for x in club_names] #stripping the file extention from the file names
fest_names = [x for x in club_names if x in festivals]
club_names = [x for x in club_names if x not in festivals]

## Functions
These functions take a list of DataFrames that were made from the .xlsx files made from scraping Resident Advisor, and creates new ones which are a count of the number of times an artist's name appears in an event lineup.

In [4]:
def clean_and_reshape(dfs,clubs):
    list_of_dfs = []
    for i in range(len(dfs)):
        artist_list = get_artists(dfs[i],clubs[i])
        artist_list = [x for x in artist_list if len(x) < 40]
        artist_list = [x for x in artist_list if len(x) > 2]
        count_dict = count(artist_list)
        count_df = pd.DataFrame(count_dict,index=[0]).melt().rename(columns={'variable':'artist','value':clubs[i]}).set_index('artist',drop=True)
        list_of_dfs.append(count_df)
    return list_of_dfs

#creating a list of artists
def get_artists(df,club):
    temp = []
    artist_list = []
    for lineup in df['lineup']:
        case = [artist.lower() for artist in lineup.split(', ')]
        temp.extend(case)
    for i in range(len(temp)):
        temp[i] = re.sub('\([^)]*\)',"",temp[i])
        temp[i] = re.sub('(live)',"",temp[i])
        temp[i] = temp[i].strip("][./)('")
        if not re.search("([0-9])",temp[i]) and temp[i] != ' ':
            artist_list.append(temp[i])
        else:
            continue
    return clean_list(artist_list,club)

def clean_list(artists,club):
    removal_list = ['saturday','tba','room','resident','students','b2b','main','august',
                    'friday','sunday','monday','tuesday','wednesday','thursday','️ibiza',
                    'dress code','installed','dancing','floor','😇','⭐',':','#','&',
                    'about','globus','bar','club','säule','terrace','garten','djs',
                    '➜','-','?','guest','tresor','panorama','one','two','more','wood',
                    '▪','■','╚●','new york','berlin','spain','london','amsterdam','•',
                    '▶︎', '◇', '○','concrete','–','horoom','records','bassiani','basement']
    for i in range(len(artists)):
        for item in removal_list:
            if item in artists[i] or item == club:
                artists[i] = artists[i].replace(item,'').strip()
    artists = [x for x in artists if x != ""]
    artists = [x for x in artists if 'line up' not in x]
    artists = [x for x in artists if x not in club_names]
    artists = [x for x in artists if x != 'the']
    artists = [x for x in artists if 'hosted' not in x]
    return artists

def count(artists):
    artist_count = {}
    distinct_artist_list = []
    for artist in artists:
        if artist not in distinct_artist_list:
            distinct_artist_list.append(artist)
            artist_count[artist] = 1
        else:
            artist_count[artist] += 1
    return artist_count

## Lists of DataFrames
Using the clean_and_reshape function, a new lists of DataFrames are created for each city. Each DataFrame is a club.

In [5]:
berlin_list = clean_and_reshape([df_list[1],df_list[8],df_list[10],df_list[15]],
                           ['about blank','watergate','berghain','tresor'])
amsterdam_list = clean_and_reshape([df_list[14],df_list[7],df_list[20],df_list[17]],
                                  ['de school','shelter','radion','warehouse elementenstraat'])
london_list = clean_and_reshape([df_list[23],df_list[5],df_list[21],df_list[0]],
                               ['fabric','corsica studios','phonox','xoyo'])
ibiza_list = clean_and_reshape([df_list[12],df_list[9],df_list[13]],
                              ['dc10','ushuaia','amnesia'])
paris_list = clean_and_reshape([df_list[18],df_list[22]],
                              ['badaboum','concrete'])

## Concat club DataFrames into one city DataFrame
Here is where the city's DataFrame is created. A new column is made to show the total number of appearances an artist made in that city, and all NaN values are turned into zeroes.

In [6]:
berlin_df = pd.concat(berlin_list,axis=1)
berlin_df['berlin_total'] = berlin_df.sum(axis=1)
berlin_df.fillna(0,inplace=True)

amsterdam_df = pd.concat(amsterdam_list,axis=1)
amsterdam_df['amsterdam_total'] = amsterdam_df.sum(axis=1)
amsterdam_df.fillna(0,inplace=True)

london_df = pd.concat(london_list,axis=1)
london_df['london_total'] = london_df.sum(axis=1)
london_df.fillna(0,inplace=True)

ibiza_df = pd.concat(ibiza_list,axis=1)
ibiza_df['ibiza_total'] = ibiza_df.sum(axis=1)
ibiza_df.fillna(0,inplace=True)

paris_df = pd.concat(paris_list,axis=1)
paris_df['paris_total'] = paris_df.sum(axis=1)
paris_df.fillna(0,inplace=True)

## Joining all cities
Using an outer join, I merged all of the cities together into one DataFrame. The NaN values again were turned to zeroes, and a new column (total_appearances) is made that adds up all of the artists' total appearances in each city.

In [7]:
all_cities_df = berlin_df.join([london_df,amsterdam_df,ibiza_df,paris_df],how='outer')

In [8]:
all_cities_df.fillna(0,inplace=True)
all_cities_df.head(10)

,about blank,watergate,berghain,tresor,berlin_total,fabric,corsica studios,phonox,xoyo,london_total,...,radion,warehouse elementenstraat,amsterdam_total,dc10,ushuaia,amnesia,ibiza_total,badaboum,concrete,paris_total
akmê,34.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alex.do,10.0,1.0,2.0,0.0,13.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
charlie smooth,13.0,1.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
compadre blanquistador,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
courtesy,2.0,0.0,5.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
diwa,26.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
diva xs,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dr. rubinstein,11.0,0.0,20.0,0.0,31.0,0.0,3.0,0.0,0.0,3.0,...,1.0,1.0,10.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
emily,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
erwan,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
all_cities_df['total_appearances'] = all_cities_df['berlin_total'] + all_cities_df['london_total'] + all_cities_df['amsterdam_total'] + all_cities_df['ibiza_total'] + all_cities_df['paris_total']

In [10]:
all_cities_df.reset_index(inplace=True)

In [11]:
all_cities_df.rename(columns={'index':'artist_name'},inplace=True)

In [13]:
all_cities_df.sort_values(by='total_appearances',ascending=False).head(40)

,artist_name,about blank,watergate,berghain,tresor,berlin_total,fabric,corsica studios,phonox,xoyo,...,warehouse elementenstraat,amsterdam_total,dc10,ushuaia,amnesia,ibiza_total,badaboum,concrete,paris_total,total_appearances
17173,raymundo rodriguez,0.0,0.0,0.0,0.0,0.0,0.0,254.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,254.0
3842,aurora,1.0,0.0,0.0,219.0,220.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,221.0
14250,jacob husley,0.0,0.0,0.0,0.0,0.0,123.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123.0
14251,peter pixzel,0.0,0.0,0.0,0.0,0.0,123.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123.0
10118,miss italia,0.0,0.0,0.0,120.0,120.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0
8341,craig richards,0.0,0.0,1.0,0.0,1.0,100.0,0.0,0.0,0.0,...,2.0,2.0,7.0,0.0,5.0,12.0,0.0,1.0,1.0,116.0
14206,terry francis,0.0,0.0,0.0,0.0,0.0,103.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,104.0
15696,chris stanford,0.0,0.0,0.0,0.0,0.0,1.0,102.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103.0
15758,haai,0.0,0.0,0.0,0.0,0.0,1.0,0.0,91.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,96.0
8344,seth troxler,0.0,0.0,4.0,0.0,4.0,12.0,0.0,0.0,0.0,...,3.0,8.0,60.0,1.0,6.0,67.0,1.0,3.0,4.0,95.0


In [107]:
all_cities_df.to_csv('ra_events_artist_count.csv',header=all_cities_df.columns)